In [ ]:
import pandas as pd
import numpy as np
from scipy.optimize import linprog 
from numpy.linalg import solve
import urllib.request
import os

### Downloading and parsing data

In [ ]:
if not os.path.exists('data'):
    os.makedirs('data')

url = "http://www.nordpoolspot.com/globalassets/marketdata-excel-files/consumption-prognosis_2019_hourly.xls" 
urllib.request.urlretrieve(url, 'data/consumption.xls');

url2 = "http://www.nordpoolspot.com/globalassets/marketdata-excel-files/wind-power-dk-prognosis_2019_hourly.xls" 
urllib.request.urlretrieve(url2, 'data/wind-prognosis.xls');

consumptionProg_raw_data = pd.read_html('data/consumption.xls')[0].values
consumptionProg_df = pd.DataFrame(consumptionProg_raw_data)
consumptionProg_df.columns = ['Dates', 'Hours', 'NO', 'SE', 'FI', 'DK1', 'DK2', 'EE', 'LV', 'LT']
consumptionProg_df = consumptionProg_df[['Dates', 'Hours', 'DK1', 'DK2']]
consumptionProg_df['Hours'] = consumptionProg_df['Hours'].str.slice(stop=2)
consumptionProg_df['ts'] = pd.to_datetime(consumptionProg_df['Dates'] + consumptionProg_df['Hours'] + ':00', format='%d-%m-%Y%H:%M')
consumptionProg_df = consumptionProg_df[consumptionProg_df['ts'] < pd.to_datetime('2019-02-01')]

## Including Import of DK1 -> Norway 200MW
consumptionProg_df['DK1'] = consumptionProg_df['DK1'] + 200

## Including Export of DK1 -> Germany 100MW
for i in consumptionProg_df.index:
    if (consumptionProg_df['ts'][i].time() >= pd.to_datetime('07:00:00').time()) & (windProg_df['ts'][i].time() < pd.to_datetime('14:00:00').time()):
        consumptionProg_df['DK1'][i] = consumptionProg_df['DK1'][i] - 100
        
## Including Import of DK2 -> Sweden 150MW
for i in consumptionProg_df.index:
    if (consumptionProg_df['ts'][i].time() >= pd.to_datetime('07:00:00').time()) & (windProg_df['ts'][i].time() < pd.to_datetime('14:00:00').time()):
        consumptionProg_df['DK2'][i] = consumptionProg_df['DK2'][i] + 150

windProg_raw_data = pd.read_html('data/wind-prognosis.xls')[0].values
windProg_df = pd.DataFrame(windProg_raw_data)
windProg_df.columns = ['Dates', 'Hours', 'DK1', 'DK2']
windProg_df['Hours'] = windProg_df['Hours'].str.slice(stop=2)
windProg_df['ts'] = pd.to_datetime(windProg_df['Dates'] + windProg_df['Hours'] + ':00', format='%d-%m-%Y%H:%M')
windProg_df = windProg_df[windProg_df['ts'] < pd.to_datetime('2019-02-01')]

windProg_df['WW1'] = windProg_df['DK1']*(1/4)
windProg_df['WW2'] = windProg_df['DK1']*(3/4)
windProg_df['EW1'] = windProg_df['DK2']*(1/3)
windProg_df['EW2'] = windProg_df['DK2']*(2/3)

supply_all = np.array([[400, 330, 345, 390, 510, 1000, 900, 1200, 320, 360, 400, 350, 730, 630, 800],
                       [70, 64, 153, 82, 89, 25, 250, 19, 43, 39, 36, 31, 5, 10, 250]])

supply_all_df = pd.DataFrame(supply_all.T).rename(columns={0:'ID', 1:'Supply', 2:'Cost'})

supply_restricted= np.array([[400, 330, 345, 390, 510, 900, 320, 360, 400, 350, 730, 630, 800],
                             [70, 64, 153, 82, 89, 250, 43, 39, 36, 31, 5, 10, 250]])

supply_restricted_df = pd.DataFrame(supply_restricted.T).rename(columns={0:'ID', 1:'Supply', 2:'Cost'})


In [ ]:
for i in windProg_df.index:
    
    power_d = consumptionProg_df.values[i,2:4]

    if (windProg_df['ts'][i].time() >= pd.to_datetime('05:00:00').time()) & (windProg_df['ts'][i].time() < pd.to_datetime('22:00:00').time()):
        power_g = np.concatenate((supply_restricted[0], windProg_df.values[i,5:]))
        cost_g = np.concatenate((supply_restricted[1], [0 , 0, 0, 0]))
    else:
        power_g = np.concatenate((supply_all[0], windProg_df.values[i,5:]))
        cost_g = np.concatenate((supply_restricted[1], [0 , 0, 0, 0]))

    print(windProg_df['ts'][i], '- Units: ', power_g.shape[0])
    print('Pg: ', power_g)
    print('Cg: ', cost_g)
    print('Pd: ', power_d)

### Standard Model

In [ ]:
A_eq = np.array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., 1.,  1., 
                  -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1., -1.]])
b_eq = 0

A = np.identity(A_eq.size)

power_g = np.array([120, 50, 200, 400, 60, 50, 60, 100, 70, 50, 70, 45, 50, 60, 50]) 
power_d = np.array([250, 300, 120, 80, 40, 70, 60, 45, 30, 35, 25, 10]) 
b = np.array([np.concatenate((power_g, power_d))]).T

cost_g = np.array([0, 0, 15, 30, 32.5, 34, 36, 37.5, 39, 40, 60, 70, 100, 150, 200])
cost_d = -np.array([200, 110, 100, 90, 85, 75, 65, 40, 38, 31, 24, 16])
c = np.concatenate((cost_g, cost_d))

res_std = linprog(c, A_eq=A_eq, b_eq = b_eq, A_ub=A, b_ub=b, bounds=(0, None))
print('Generation schedules:', res_std.x[:15])
print('Demand schedules:', res_std.x[15:])

### Dual

In [ ]:
a1 = np.ones(15)
a2 = -np.ones(12)
a3 = np.array([np.hstack((a1,a2))]).T
a4 = -np.identity(27)

A_tilde = np.concatenate((a4,a3),axis=1)

b_tilde = np.concatenate((cost_g, cost_d))

c_tilde = np.append(-b, np.array([0]))

res_dual = linprog(-c_tilde, A_ub=A_tilde, b_ub=b_tilde)
print('Cost:', res_dual.x[-1])

### Suppliers Dictionary

In [ ]:
suppliers_dict = {'G1' : 'FlexiGas',
                 'G2' : 'FlexiGas',
                 'G3' : 'FlexiGas',
                 'G4' : 'Peako',
                 'G5' : 'Peako',
                 'G6' : 'Nuke22',
                 'G7' : 'CoalAtLast',
                 'G8' : 'Nuke22',
                 'G9' : 'RoskildeCHP',
                 'G10' : 'RoskildeCHP',
                 'G11' : 'Avedovre',
                 'G12' : 'Avedovre',
                 'G13' : 'BlueWater',
                 'G14' : 'BlueWater',
                 'G15' : 'CoalAtLast'}